## Summary
Master post for the blog series that holds all the links related to making web service calls to Eoddata.com

* http://stackoverflow.com/a/17378332/893766
* http://stackoverflow.com/a/1912483/893766
* hidden password entry: https://docs.python.org/2/library/getpass.html

## Setup

In [19]:
from getpass import getpass
import requests as r
import xml.etree.cElementTree as etree

ws = 'http://ws.eoddata.com/data.asmx'
ns='http://ws.eoddata.com/Data'
session = r.Session()

In [2]:
username = getpass()

········


In [3]:
password = getpass()

········


## Login()

In [4]:
call = 'Login'
url = '/'.join((ws, call))

payload = {'Username': username, 'Password': password}

response = session.get(url, params=payload, stream=True)

if response.status_code == 200:
    root = etree.parse(response.raw).getroot()

### Data inspection (Login)

In [5]:
dir(root)

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'extend',
 'find',
 'findall',
 'findtext',
 'get',
 'getchildren',
 'getiterator',
 'insert',
 'items',
 'iter',
 'iterfind',
 'itertext',
 'keys',
 'makeelement',
 'remove',
 'set']

In [8]:
for child in root.getchildren():
    print (child.tag, child.attribute)

In [7]:
for item in root.items():
    print (item)

('Message', 'Login Successful')
('DataFormat', 'MSAI')
('Suffix', 'true')
('Token', '0185587J4T68')
('Header', 'false')


In [9]:
for key in root.keys():
    print (key)

Message
DataFormat
Suffix
Token
Header


In [10]:
print (root.get('Message'))
print (root.get('Token'))
print (root.get('DataFormat'))
print (root.get('Header'))
print (root.get('Suffix'))

Login Successful
0185587J4T68
MSAI
false
true


### Get data  (Login)

In [11]:
token = root.get('Token')

## ExchangeList()

In [12]:
call = 'ExchangeList'
kwargs = {'Token': token,}
pattern = ".//{%s}EXCHANGE"

url = '/'.join((ws, call))
resp = s.get(url, params=kwargs, stream=True)

if resp.status_code == 200:
    root = etree.parse(resp.raw).getroot()

### Data inspection  (ExchangeList)

In [13]:
for item in root.items():
    print (item)

('Message', 'Success')
('Date', '2017-02-23T06:36:50.876086-06:00')
('Source', 'Data.ExchangeList')


### Gather elements (Exchangelist)

In [22]:
elements = root.findall(pattern % (ns))

### Save to file (ExchangeList)

In [ ]:
with open('./data/exchanges.csv', 'w') as f:
    for element in elements:
        f.write('"%s"\n' % '","'.join(element.attrib.values()))

### Get data (ExchangeList)

In [23]:
exchanges = tuple(sorted(element.get('Code') for element in elements))
exchanges

('AMEX',
 'AMS',
 'ASX',
 'BRU',
 'BSE',
 'CBOT',
 'CFE',
 'CME',
 'COMEX',
 'EUREX',
 'FOREX',
 'HKEX',
 'INDEX',
 'KCBT',
 'LIFFE',
 'LIS',
 'LSE',
 'MGEX',
 'MLSE',
 'NASDAQ',
 'NSE',
 'NYBOT',
 'NYMEX',
 'NYSE',
 'NZX',
 'OTCBB',
 'PAR',
 'SGX',
 'TSX',
 'TSXV',
 'USMF',
 'WCE')

In [27]:
print (root)

<Element '{http://ws.eoddata.com/Data}RESPONSE' at 0x7f7261fa0f98>


In [28]:
for element in root.iter():
    print("%s|%s" % (element.tag, element.attrib))

{http://ws.eoddata.com/Data}RESPONSE|{'Source': 'Data.ExchangeList', 'Message': 'Success', 'Date': '2017-01-19T00:02:47.7793906-06:00'}
{http://ws.eoddata.com/Data}EXCHANGES|{}
{http://ws.eoddata.com/Data}EXCHANGE|{'Country': 'US', 'IntradayStartDate': '2008-01-01T00:00:00', 'Declines': '897', 'LastTradeDateTime': '2017-01-18T16:59:59', 'TimeZone': 'Eastern Standard Time', 'Currency': 'USD', 'Suffix': '', 'HasIntradayProduct': 'true', 'Code': 'AMEX', 'IsIntraday': 'true', 'Name': 'American Stock Exchange', 'Advances': '3011'}
{http://ws.eoddata.com/Data}EXCHANGE|{'Country': 'AU', 'IntradayStartDate': '0001-01-01T00:00:00', 'Declines': '494', 'LastTradeDateTime': '2017-01-19T00:00:00', 'TimeZone': 'AUS Eastern Standard Time', 'Currency': 'AUD', 'Suffix': 'AX', 'HasIntradayProduct': 'false', 'Code': 'ASX', 'IsIntraday': 'false', 'Name': 'Australian Securities Exchange', 'Advances': '14665'}
{http://ws.eoddata.com/Data}EXCHANGE|{'Country': 'IN', 'IntradayStartDate': '0001-01-01T00:00:00',

In [29]:
print(etree.tostring(root))

b'<ns0:RESPONSE xmlns:ns0="http://ws.eoddata.com/Data" Date="2017-01-19T00:02:47.7793906-06:00" Message="Success" Source="Data.ExchangeList">\n  <ns0:EXCHANGES>\n    <ns0:EXCHANGE Advances="3011" Code="AMEX" Country="US" Currency="USD" Declines="897" HasIntradayProduct="true" IntradayStartDate="2008-01-01T00:00:00" IsIntraday="true" LastTradeDateTime="2017-01-18T16:59:59" Name="American Stock Exchange" Suffix="" TimeZone="Eastern Standard Time" />\n    <ns0:EXCHANGE Advances="14665" Code="ASX" Country="AU" Currency="AUD" Declines="494" HasIntradayProduct="false" IntradayStartDate="0001-01-01T00:00:00" IsIntraday="false" LastTradeDateTime="2017-01-19T00:00:00" Name="Australian Securities Exchange" Suffix="AX" TimeZone="AUS Eastern Standard Time" />\n    <ns0:EXCHANGE Advances="4457" Code="BSE" Country="IN" Currency="INR" Declines="0" HasIntradayProduct="false" IntradayStartDate="0001-01-01T00:00:00" IsIntraday="false" LastTradeDateTime="2017-01-18T00:00:00" Name="Bombay Stock Exchange" 

In [30]:
dir(exchange.attrib)

NameError: name 'exchange' is not defined

In [65]:
s.close()

AttributeError: 'str' object has no attribute 'close'